# Programação Linear

Aqui está um tutorial para instalar o solver e escrever programas lineares e inteiros:

Parte 1: https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Linear_Programming.ipynb

Parte 2: https://github.com/IBMDecisionOptimization/docplex-examples/blob/master/examples/mp/jupyter/tutorials/Beyond_Linear_Programming.ipynb

# Exercício 1 - Modelagem

  Considere a seguinte tabela de informação nutricional de vários
  tipos de alimentos (os preços, calorias, gorduras, proteínas e
  carboidratos são por porção):

  <style type="text/css">
.tg  {border-collapse:collapse;border-spacing:0;}
.tg td{font-family:Arial, sans-serif;font-size:14px;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg th{font-family:Arial, sans-serif;font-size:14px;font-weight:normal;padding:10px 5px;border-style:solid;border-width:1px;overflow:hidden;word-break:normal;border-color:black;}
.tg .tg-0pky{border-color:inherit;text-align:left;vertical-align:top}
.tg .tg-0lax{text-align:left;vertical-align:top}
</style>
<table class="tg">
  <tr>
    <th class="tg-0pky">Alimento</th>
    <th class="tg-0pky">Preço</th>
    <th class="tg-0pky">Calorias (cal)</th>
    <th class="tg-0pky">Gorduras (g)</th>
    <th class="tg-0pky">Proteínas (g)</th>
    <th class="tg-0lax">Carboidratos (g)</th>
  </tr>
  <tr>
    <td class="tg-0lax">Batata crua</td>
    <td class="tg-0lax">0.14</td>
    <td class="tg-0lax">23</td>
    <td class="tg-0lax">0.1</td>
    <td class="tg-0lax">0.6</td>
     <td class="tg-0lax">6</td>
  </tr>
  <tr>
    <td class="tg-0lax">Batata assada</td>
    <td class="tg-0lax">0.12</td>
    <td class="tg-0lax">171</td>
    <td class="tg-0lax">0.2</td>
    <td class="tg-0lax">3.7</td>
    <td class="tg-0lax">30</td>
  </tr>
  <tr>
    <td class="tg-0lax">Pão</td>
    <td class="tg-0lax">0.20</td>
    <td class="tg-0lax">65</td>
    <td class="tg-0lax">0.0</td>
    <td class="tg-0lax">2.2</td>
    <td class="tg-0lax">13</td>
  </tr>
  <tr>
    <td class="tg-0lax">Cheddar</td>
    <td class="tg-0lax">0.75</td>
    <td class="tg-0lax">112</td>
    <td class="tg-0lax">9.3</td>
    <td class="tg-0lax">7.0</td>
    <td class="tg-0lax">0</td>
  </tr>
  <tr>
    <td class="tg-0lax">Amendoim</td>
    <td class="tg-0lax">0.15</td>
    <td class="tg-0lax">188</td>
    <td class="tg-0lax">16.0</td>
    <td class="tg-0lax">7.7</td>
    <td class="tg-0lax">2</td>
  </tr>
</table>
  

  Você deve montar uma dieta decidindo quantas porções dos alimentos
  comprar de modo que o custo total pago seja minimizado e que
  os seguintes requisitos sejam cumpridos:
 
  - O número de calorias de ser pelo menos $2000$,
  - o total de gordura deve ser pelo menos $50$g
  - o total de proteínas deve ser pelo menbos $100$g,
  - o total de carboidratos deve ser pelo menos $250$g.
  
  O número de porções pode ser um número fracionário (por exemplo,
  você poderia pegar $2.156$ porções de batata).
    
  Escreva um programa linear e resolva usando o solver.

In [3]:
import sys
from docplex.mp.model import Model
m = Model(name='Dieta')

batatacrua = m.continuous_var(name='batatacrua')
batataassada = m.continuous_var(name='batataassada')
pao = m.continuous_var(name='pao')
cheddar = m.continuous_var(name='cheddar')
amendoim = m.continuous_var(name='amendoim')

# write constraints
# constraint #1:
m.add_constraint(23*batatacrua + 171*batataassada + 65*pao + 112*cheddar + 188*amendoim >= 2000)

# constraint #2:
m.add_constraint(0.1*batatacrua + 0.2*batataassada + 9.3*cheddar + 16*amendoim >= 50)

# constraint #3:
m.add_constraint(0.6*batatacrua + 3.7*batataassada + 2.2*pao + 7*cheddar + 7.7*amendoim >= 100)

# constraint #4:
m.add_constraint(6*batatacrua + 30*batataassada + 13*pao + 2*amendoim >= 250)

m.minimize(0.14*batatacrua + 0.12*batataassada + 0.2*pao + 0.75*cheddar + 0.15*amendoim)

m.print_information()

s = m.solve()

print("\nResultado: \n")

m.print_solution()


Model: Dieta
 - number of variables: 5
   - binary=0, integer=0, continuous=5
 - number of constraints: 4
   - linear=4
 - parameters: defaults

Resultado: 

objective: 2.318
  batataassada=7.715
  amendoim=9.280


# Exercício 2 - Set cover

## Problema: **Set Cover** 

Entrada: $m,n, (S_1,..., S_n) , w$, onde $\bigcup_{i=1}^n S_i = \{1,2,\dotsc, m\}$ e $w$ é um vetor de custos positivos de tamanho $n$.

Objetivo: Encontrar $I\subseteq \{1,\dotsc n\}$ tal que $\bigcup_{i\in I}S_i = \{1,\dotsc, m\}$ de modo a minimizar $\sum_{i\in I}w_i$.

In [9]:
# Gerador aleatório

import random as rd

def geraAleatorio(p, mincost, maxcost,n, m): 
    # p deve ser um numero de 0 a 1. Os conjuntos terao tamanho esperado p*m
    # mincost = menor custo possivel, maxcost = maior custo possivel
    w = [rd.randint(mincost,maxcost) for i in range(0, n)]
    S = []
    inc = m*[0]
    for i in range (0,n):
        C = []
        for j in range(1,m+1):
            if p > rd.random():
                C.append(j)
                inc[j-1] = 1
        S.append(C)
    for j in range(0,m):
        if inc[j]==0:
            i = rd.randint(0,n-1)
            S[i].append(j+1)
    # checagem
    chec = m*[0]
    for i in range(0,n):
        for j in S[i]:
            chec[j-1]=1
    for j in range(0,m):
        if chec[j]==0:
            print("Erro",str(j+1),"nao foi incluido em nenhum conjunto")
    return (m,n,S,w)
   
# Exemplo de uso
m,n,S,w = geraAleatorio(0.2, 1,5, 10, 20)

print(m,"\n",n,"\n",S,"\n",w)

20 
 10 
 [[10, 13, 5], [4, 8, 16, 17, 6], [1, 7, 8, 13, 16, 18], [7, 9, 11, 12, 19, 20], [10, 13, 14, 17], [1, 2, 4, 8, 16, 18], [8, 10, 14], [3, 4, 7, 9, 14, 16], [4, 10, 13, 15, 20], [1, 17]] 
 [4, 2, 4, 3, 3, 1, 4, 2, 5, 2]


In [10]:
def set_cover_int(m,n,S,w):
    
    scm = Model(name='SetCover')
 
    for i in n:
        x[i] = scm.integer_var(name='x_{0}')

    for i in n:
        scm.add_constraint((x[i]) >= 1)
    
    C = {(i): scm.integer_var(name='C_{0}'.format(i)) for i in n}
    
    #{(x[S[i][j]] for j in len(S[i])) : scm.integer_var(name='C_{0}'.format(i)) for i in n}
    
    for i in n:
        scm.add_constraint(C[i] == scm.sum(x[j]) for j in len(S[i]))
    
    scm.minimize(scm.sum(C[i]*w[i] for i in n))
    
    print("\n")
    
    scm.print_information()
    
    print("\nResultado:\n")
    
    scm.print_solution()
    

In [11]:
set_cover_int(m,n,S,w)

TypeError: 'int' object is not iterable

In [38]:
def set_cover_relax(m,n, S, w):
    
    scm = Model(name='SetCover')
    
    x = {(i): scm.continuous_var(name='x_{0}'.format(i)) for i in n}
        
    for i in n:
        scm.add_constraint((x[i]) >= 1)
    
    C = {(x[S[i][j]] for j in len(S[i])) : scm.continuous_var(name='C_{0}'.format(i)) for i in n}
    
    scm.minimize(scm.sum(C[i]*w[i] for i in n))
    
    print("\n")
    
    scm.print_information()
    
    print("\nResultado:\n")
    
    scm.print_solution()
    

(a) Faça uma função `set_cover_int(m,n, S, w)` que cria e resolve o programa **inteiro** para Set Cover (use o solver). `S` é uma lista formada de $n$ listas e $w$ é uma lista de tamanho $n$. (Veja o que o gerador aleatório devolve).

(b) Faça uma função `set_cover_relax(m,n, S, w)` que cria e resolve o programa linear que é uma relaxação para Set Cover.

(c) Faça testes com as funções acima e o gerador aleatório verificando a diferença de tempo de execução e dos valores ótimos encontrados. Deixe as suas execuções rodadas aqui no notebook. De preferência, inclua um gráfico com os resultados obtidos. Escreva um parágrafo (em Markdown) analisando os resultados obtidos.

# Exercício 3 - Algoritmo probabilístico para Set Cover

Implemente a estratégia probabilística vista em sala utilizando a função implementada em 2(b).

Faça testes utilizando o gerador aleatório e verificando diferença de tempo de execução do algoritmo com a função de 2(a) e os valores ótimos. Deixe as suas execuções rodadas aqui no notebook. De preferência, inclua um gráfico com os resultados obtidos. Escreva um parágrafo (em Markdown) analisando os resultados obtidos.